In [53]:
import json
import pandas as pd
with open("../medical_data/Corona2.json", "r") as f:
    data = json.load(f)

examples = data["examples"]
df_main = pd.DataFrame(examples)
df_main.head()

,id,content,metadata,annotations,classifications
0,18c2f619-f102-452f-ab81-d26f7e283ffe,While bismuth compounds (Pepto-Bismol) decreas...,{},[{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed'...,[]
1,487c93e3-0d45-4088-a378-cf3a01c8953d,"Diarrhea, also spelled diarrhoea, is the condi...",{},[{'id': '28601a42-c8a9-44e2-aeea-8939cb1db1a9'...,[]
2,d5056874-895a-4a7f-9e0f-828d414d65d9,Antiretroviral therapy (ART) is recommended fo...,{},[{'id': '0a4b4cc2-3a07-4817-8ccc-8fa1f945573d'...,[]
3,20c792c7-0c4b-42d0-8127-0e04113db384,The following drugs are considered as DMARDs: ...,{},[{'id': '0e370ff5-fee6-45e1-adcb-d5f00fa33b0d'...,[]
4,f5359e0d-4d4a-4707-95a3-4c627fc4a83b,"The goals of treatment are to reduce pain, dec...",{},[{'id': '10411ff5-e343-40bf-95d5-39cd1cb16060'...,[]


In [44]:
df_main["annotations"][0]

[{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
  'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
  'end': 371,
  'start': 360,
  'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
  'tag_name': 'Medicine',
  'value': 'Diosmectite',
  'correct': None,
  'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
    'annotator_id': 1,
    'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
    'name': 'Ashpat123',
    'reason': 'exploration'}],
  'model_annotations': []},
 {'id': '145f62b1-bbf5-42f1-8ad5-9c7e08337bf0',
  'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
  'end': 408,
  'start': 383,
  'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
  'tag_name': 'Medicine',
  'value': 'aluminomagnesium silicate',
  'correct': None,
  'human_annotations': [{'timestamp': '2020-03-21T00:24:43.692000Z',
    'annotator_id': 1,
    'tagged_token_id': '145f62b1-bbf5-42f1-8ad5-9c7e08337bf0',
    'name': 'Ashpat123',
    'reason': 'exploration'}],
  'model_annotations':

In [13]:
print((df_main["metadata"] == {}).all())
df_main["classifications"].apply(lambda x: x == []).all()

True


True

In [18]:
df_main = df_main.drop(columns=['metadata', 'classifications'])
df_main.columns

Index(['id', 'content', 'annotations'], dtype='object')

# TODO:
* read https://mageswaran1989.medium.com/how-to-build-custom-ner-huggingface-dataset-for-receipts-and-train-with-huggingface-transformers-6c954b84473c
* get all tags
* get tokens

## Refromat tags

In [54]:
from tqdm import tqdm
import typing

names_to_tags_map = {
    'Medicine' : "MED",
    'MedicalCondition' : "MEDC",
    'Pathogen' : "PAT"
}
tags_to_id = {"O": 0}
bedin_prefix = "B-"
inter_prefix = "I-"
end_prefix = "E-"
max_id = 0

MODE = typing.Literal['s', 'm', 'e']

def update_tag(anno : dict, where : MODE, part: str):
    if where == "s":
        anno['tag_name'] = bedin_prefix + names_to_tags_map[anno['tag_name']]
    elif where == "e":
        anno['tag_name'] = end_prefix + names_to_tags_map[anno['tag_name']]
    else:
        anno['tag_name'] = inter_prefix + names_to_tags_map[anno['tag_name']]
    anno["value"] = part


for i, sample in tqdm(enumerate(df_main["annotations"])):
    new_sample = []
    for ann in sample:
        parts = ann['value'].split(" ")
        for i, part in enumerate(parts):
            new_ann = ann.copy()
            if i == 0:
                update_tag(new_ann, 's', part)
            elif i == len(parts) - 1:
                update_tag(new_ann, 'e', part)
            else:
                update_tag(new_ann, 'm', part)
            new_sample.append(new_ann)
        if ann['tag_name'] not in tags_to_id:
            max_id += 1
            tags_to_id[ann['tag_name']] = max_id

    df_main["annotations"].at[i] = new_sample
        


tags_to_id

2it [00:31, 15.63s/it]


KeyError: 'B-MEDC'

## train_val_split

In [ ]:
from sklearn.model_selection import train_test_split
X = df_main["content"]
y = df_main["annotations"]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
print("Train size:", len(X_train))
print("Test size:", len(X_test))

Train size: 21
Test size: 10


In [ ]:
X_train.tolist()[0]

"The goals of treatment are to reduce pain, decrease inflammation, and improve a person's overall functioning.[5] This may be helped by balancing rest and exercise, the use of splints and braces, or the use of assistive devices.[1][6][7] Pain medications, steroids, and NSAIDs are frequently used to help with symptoms.[1] Disease-modifying antirheumatic drugs (DMARDs), such as hydroxychloroquine and methotrexate, may be used to try to slow the progression of disease.[1] Biological DMARDs may be used when disease does not respond to other treatments.[8] However, they may have a greater rate of adverse effects.[9] Surgery to repair, replace, or fuse joints may help in certain situations.[1] Most alternative medicine treatments are not supported by evidence.[10][11]"

In [ ]:
from typing import List
import re
from string import punctuation
def yield_examples(texts : list, annotations: List[dict]):
    for sent, anns in zip(texts, annotations):
        sent = re.sub("\[\d*\]", "", sent)
        nopunc = [char for char in sent if char not in punctuation]
        cleared = ''.join(nopunc)
        value_tag = { ann["value"] : ann['tag_name'] for ann in anns}
        tags = []
        tokens = []
        print(cleared)
        for token in cleared.split(" "):
            tag = value_tag.get(token, "0")
            tags.append(tags_to_id[tag])
            tokens.append(token)
        
        print(tokens)
        print(tags)
        break
yield_examples(X_train.tolist(), y_train.tolist())

The goals of treatment are to reduce pain decrease inflammation and improve a persons overall functioning This may be helped by balancing rest and exercise the use of splints and braces or the use of assistive devices Pain medications steroids and NSAIDs are frequently used to help with symptoms Diseasemodifying antirheumatic drugs DMARDs such as hydroxychloroquine and methotrexate may be used to try to slow the progression of disease Biological DMARDs may be used when disease does not respond to other treatments However they may have a greater rate of adverse effects Surgery to repair replace or fuse joints may help in certain situations Most alternative medicine treatments are not supported by evidence
['The', 'goals', 'of', 'treatment', 'are', 'to', 'reduce', 'pain', 'decrease', 'inflammation', 'and', 'improve', 'a', 'persons', 'overall', 'functioning', 'This', 'may', 'be', 'helped', 'by', 'balancing', 'rest', 'and', 'exercise', 'the', 'use', 'of', 'splints', 'and', 'braces', 'or', 

In [ ]:
import pickle
train = X_train.tolist(), y_train.tolist()
val = X_test.tolist(), y_test.tolist()
with open("train.pkl", "wb") as f:
    pickle.dump(train, f)
with open("val.pkl", "wb") as f:
    pickle.dump(val, f)

In [ ]:
from datasets import GeneratorBasedBuilder
import datasets
from pickle import Unpickler

class MedicalNerDataset(GeneratorBasedBuilder):

    def __init__(self, train_pkl="train.pkl", val_pkl="val.pkl", **kwargs) -> None:
        super().__init__(**kwargs)
        self.train_pkl = train_pkl
        self.val_pkl = val_pkl
        self._ner_mapping = {'0': 0, 'Medicine': 1, 'MedicalCondition': 2, 'Pathogen': 3}

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": self.train_pkl}),
            datasets.SplitGenerator(name=datasets.Split.VALIDATION, gen_kwargs={"filepath": self.val_pk}),
        ]

    def _info(self):
        return datasets.DatasetInfo(
            description="",
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "tokens": datasets.Sequence(datasets.Value("string")),
                    "ner_tags": datasets.Sequence(
                        datasets.features.ClassLabel(
                            names=sorted(list(self.self._ner_mapping.keys()))
                        )
                    )
                }
            ),
            supervised_keys=None,
            homepage="",
            citation="",
        )

    def get_tags_tokens(self, sent, anns):
        sent = re.sub("\[\d*\]", "", sent)
        nopunc = [char for char in sent if char not in punctuation]
        cleared = ''.join(nopunc)
        value_tag = { ann["value"] : ann['tag_name'] for ann in anns}
        tags = []
        tokens = []
        print(cleared)
        for token in cleared.split(" "):
            tag = value_tag.get(token, "0")
            tags.append(tags_to_id[tag])
            tokens.append(token)
        return tags,tokens

    def _generate_examples(self, filepath):
        texts, annotations = Unpickler(filepath)
        guid = -1
        for sent, anns in zip(texts, annotations):
            tags, tokens = self.get_tags_tokens(sent, anns)
            guid += 1
            yield guid, {
                "id": str(guid),
                "tokens": tokens,
                "ner_tags": tags,
            }


In [ ]:
ds = MedicalNerDataset()
ds

TypeError: <module '__main__'> is a built-in module